In [1]:
import sys;  sys.path.append('..')
from nphm_dataset import NPHMDataset
from pathlib import Path

data_path = Path('/apollo/datasets/NPHM/')
ds_train = NPHMDataset(data_path, subset='train')

In [2]:
from torch.utils.data import DataLoader

In [3]:
batch_size = 128

dl_train = DataLoader(
    ds_train,
    batch_size=batch_size,
    num_workers=8
)

In [4]:
def agg_means(means, batch_size, last_batch_size):
    return (torch.sum(means[:-1, ...], dim=0) + (last_batch_size / batch_size) * means[-1, ...])/len(means)

In [5]:
from tqdm.notebook import tqdm
import torch

pos_means = []

for v_off, pos, ids in tqdm(dl_train):
    pos = pos.to('cuda')
    pos_means.append(
        pos.mean(dim=0)
    )
    last_batch_size = len(pos)

pos_means = torch.stack(pos_means)
pos_mean = agg_means(pos_means, batch_size, last_batch_size)

  0%|          | 0/41 [00:00<?, ?it/s]

In [6]:
pos_vars = []

for v_off, pos, ids in tqdm(dl_train):
    pos = pos.to('cuda')
    pos_vars.append(
        (pos - pos_mean).pow(2).mean(dim=0)
    )
    last_batch_size = len(pos)

pos_vars = torch.stack(pos_vars)
pos_std = torch.sqrt(agg_means(pos_vars, batch_size, last_batch_size))

  0%|          | 0/41 [00:00<?, ?it/s]

In [7]:
import numpy as np

np.save('nphm_mean_vertices.npy', pos_mean.cpu().numpy())
np.save('nphm_std_vertices.npy', pos_std.cpu().numpy())